<a href="https://colab.research.google.com/github/kobymallah/AttacksonImplementationsCourseBook/blob/add-lecture1-code/Labs/Lecture1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating the problem with the AN/FGQ-1 Mixer during World War 2 and the Faraday cage countermeasure that was proposed

In [ ]:
import typing
import random
import numpy as np
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column
output_notebook()

First, we define a function that takes an integer and returns a numpy array that represents the bits of that integer (we assume that the integer is less than 2^128, because 128 bits is the size of the key)

In [ ]:
def bitfield(n):
    bits = np.array([int(digit) for digit in bin(n)[2:]])
    result = np.zeros(128)
    result[128 - bits.shape[0]:] = bits
    return np.repeat(result, 5)

Next, we define the mixer function, which has a secret key. The mixer function takes the text as an input, xor's this value with the secret key and returns: (1) a numpy array that represents the bits of the encryption and (2) the power leaks.

In [ ]:
# This mixer xor's the text and the secret key and outputs the encrypted text and the leakage
def mixer(text):
  secret_key = 0xABEF1CD9153A548222F2B5A10D60FF73

  encryption = secret_key ^ text
  encryption_as_bits = bitfield(encryption)
  noise = np.random.normal(0, .1, encryption_as_bits.shape)
  leak = encryption_as_bits + noise

  return encryption_as_bits, leak

Now, we define the faraday cage countermeasure, which isolates the mixer and prevents the electromagnetic signals from escaping.

In [ ]:
# A Faraday cage simulation. The cage surrounds the mixer and prevents 
def faraday_cage(func):
  def wrapper(*args, **kwargs):
    encrypted, leakage = func(*args, **kwargs)
    leakage = np.random.normal(0, .1, np.array(leakage).shape)
    return encrypted, leakage
  return wrapper

Here we encrypt the message with or without using a faraday cage.

In [ ]:
message = 0xC3019D348CB7ADF576BAE1DD6A25049B
data_without_faraday = mixer(message)
data_with_faraday = faraday_cage(mixer)(message)

In [ ]:
data_with_faraday[1].shape

(640,)

This is our simulation of the oscilloscope, which is simply plotting the leakage at the top and the original encrypted bits at the bottom.

In [ ]:
def oscilloscope(leakage, encrypted):
  leakage_fig = figure(plot_width=400, plot_height=400, title="Leakage")
  encrypted_fig = figure(plot_width=400, plot_height=400, title="Encrypted Bits", x_range=leakage_fig.x_range)

  x = range(leakage.shape[0])

  leakage_fig.line(x, leakage)
  encrypted_fig.line(x, encrypted)

  show(column(leakage_fig, encrypted_fig))

The oscilloscope plots when we don't use a faraday cage.

In [ ]:
oscilloscope(data_without_faraday[1], data_with_faraday[0])

The oscilloscope plots when we use a faraday cage.

In [ ]:
oscilloscope(data_with_faraday[1], data_with_faraday[0])

As described in the introduction, the scientists at Bell labs set up an antenna and they listened for an hour for radio transmissions that the Secret Service got in their office, and they gave the Secret Service their “top-secrets” analyzed messages. Here, we take the leaked radio transmissions and reconstruct the encryption bits.

In [ ]:
def get_encrypted_bits_from_leakage(leakage):
  bits = []
  threshold = 0.8
  leakage = np.reshape(leakage, (-1, 5))
  for value in leakage:
    if np.mean(value) > threshold:
      bits.extend([1] * 5)
    else:
      bits.extend([0] * 5)
  return np.array(bits)

In [ ]:
constructed_bits = get_encrypted_bits_from_leakage(data_without_faraday[1])
oscilloscope(constructed_bits, data_without_faraday[0])

As we can see, there is a perfect match between the reconstructed bits and the original encrypted bits.